### Cosc2671 Assignment 2 Group 15 


In [6]:
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from nltk.sentiment import SentimentIntensityAnalyzer
import seaborn as sns
import nltk
import praw
import datetime as dt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
# Initialize tools
tokenizer = TweetTokenizer()
tokenizer = TweetTokenizer(preserve_case=False)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [8]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    #Remove non-alphabetic characters but keep @, #, and apostrophe
    text = re.sub(r'[^a-zA-Z@\s#\']', '', text)
    # Tokenize
    tokens = tokenizer.tokenize(text)
    # Remove stopwords and short tokens; lemmatize
    cleaned = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
    return cleaned

In [9]:
# Reddit API credentials
clientid = "FgY4iejqu83Q9QpgNMX9dQ"
clientsecret = "eeSZYZUjfMIAQ5qcULOc9SSYOHghFQ"
password = "1357900jkx"
username = "Traditional-Shine402"
useragents = 'client for SNAM2024'
reddit = praw.Reddit(client_id=clientid, client_secret=clientsecret,password=password,
                           username=username,user_agent=useragents)


# Keywords and Subreddits
keywords = [
    "moon landing fake", 
    "moon landing hoax", 
    "fake moon landing", 
    "did we land on the moon", 
    "moon landing conspiracy", 
    "moon landing was staged", 
    "apollo mission fake", 
    "moon landing faked", 
    "moon hoax", 
    "apollo hoax", 
    "moon conspiracy", 
    "moon landing lies", 
    "nasa faked moon landing"]

subreddits = ["conspiracy", "moonlandingfake"]


In [10]:
#Data collection
posts = []

for subreddit in subreddits:
    for keyword in keywords:
        for submission in reddit.subreddit(subreddit).search(keyword, limit=10000):
            posts.append({
                'subreddit': subreddit,
                'title': submission.title.lower(),
                'text': submission.selftext.lower(),
                'author': str(submission.author),
                'score': submission.score,
                'created_utc': dt.datetime.fromtimestamp(submission.created_utc),
                'id': submission.id,
                'num_comments': submission.num_comments
            })

posts_df = pd.DataFrame(posts)
posts_df['cleaned_tokens'] = posts_df['text'].astype(str).apply(preprocess_text)
posts_df.to_csv("reddit_posts.csv", index=False)
print(f"Fetched {len(posts_df)} posts.")

# Fetch comments
comments_data = []

for post_id in posts_df['id']:
    if len(comments_data) > 25000:
        break
    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        comments_data.append({
            'post_id': post_id,
            'comment_body': comment.body.lower(),
            'author': str(comment.author),
            'score': comment.score,
            'created_utc': dt.datetime.fromtimestamp(comment.created_utc)
        })

comments_df = pd.DataFrame(comments_data)
comments_df['cleaned_tokens'] = comments_df['comment_body'].astype(str).apply(preprocess_text)
comments_df.to_csv("reddit_comments.csv", index=False)
print(f"Fetched {len(comments_df)} comments.")


Fetched 2849 posts.
Fetched 25047 comments.


In [11]:
posts_df

,subreddit,title,text,author,score,created_utc,id,num_comments,cleaned_tokens
0,conspiracy,did we ever land on the moon…?,,Own_Teacher3433,738,2024-08-23 22:28:45,1ezbh2v,759,[]
1,conspiracy,can you tell me why you think the moon landing...,i come in peace! i am a about to start my phd ...,Ms_Photon,863,2023-03-12 01:48:43,11olzpd,1374,"[come, peace, start, phd, astronomy, first, th..."
2,conspiracy,"if the moon landing was fake, why the enemies ...",,TimmyOTule,196,2024-09-27 22:26:50,1fqmhry,527,[]
3,conspiracy,why do people get so defensive when the 1969 m...,whenever a moon landing conspiracy is posted t...,Majin_Vegeta_,249,2024-01-03 10:27:29,18x3uai,449,"[whenever, moon, landing, conspiracy, posted, ..."
4,conspiracy,if the moon landing was faked how have other c...,"in my limited understanding of space travel, w...",5afari,383,2022-07-13 22:57:31,vy2lws,525,"[limited, understanding, space, travel, would,..."
...,...,...,...,...,...,...,...,...,...
2844,moonlandingfake,why i no longer believe the apollo moon landin...,,HelicopterJesus,3,2021-10-27 04:31:21,qgbkci,1,[]
2845,moonlandingfake,here's the truth,moon landings were faked on the moon.💀,Sea-Mark-7504,11,2023-03-04 04:55:36,11h8s9e,5,"[moon, landing, faked, moon]"
2846,moonlandingfake,bruh,i've gotten 50 new types of brain cancer from ...,YesterdayDirect8401,20,2022-11-26 00:36:44,z4dkp6,6,"[gotten, new, type, brain, cancer, came, schoo..."
2847,moonlandingfake,is it me or has the internet become the most e...,i used to use it to win all kinds of arguments...,ElectronicPresenc,12,2022-09-21 16:45:45,xjxct3,7,"[used, use, win, kind, argument, would, sound,..."


In [12]:
comments_df

,post_id,comment_body,author,score,created_utc,cleaned_tokens
0,1ezbh2v,###[meta] sticky comment\n\n[rule 2](https://w...,AutoModerator,1,2024-08-23 22:28:47,"[###meta, sticky, comment, rule, apply, replyi..."
1,1ezbh2v,i once asked a.i how to get sugar out of a gas...,dinahmoon,799,2024-08-24 01:15:41,"[asked, get, sugar, gas, tank, told, use, stra..."
2,1ezbh2v,everyone loves ai when it gives you the answer...,Binarydemons,156,2024-08-24 03:49:59,"[everyone, love, give, answer, want, hear]"
3,1ezbh2v,wow that ai has such an unbiased sounding name,before686entenz,642,2024-08-23 23:18:37,"[wow, unbiased, sounding, name]"
4,1ezbh2v,"i don't know, but china confirmed their probes...",AccordingWarning9534,322,2024-08-24 01:16:11,"[know, china, confirmed, probe, moon, rover, s..."
...,...,...,...,...,...,...
25042,6hvagj,"science olympiad in tenth grade, why?",regular_poster,1,2017-06-18 22:08:12,"[science, olympiad, tenth, grade]"
25043,6hvagj,"i'm an lpn, lol.",regular_poster,2,2017-06-18 10:19:56,"[lpn, lol]"
25044,6hvagj,>a bird doesn't escape gravity.\n\nneither doe...,EnoughNoLibsSpam,1,2017-06-18 10:53:16,"[bird, escape, gravity, neither, astronaut]"
25045,6hvagj,"as i said, theres a reason question 1 is quest...",EnoughNoLibsSpam,1,2017-06-18 09:55:54,"[said, there, reason, question, question, imme..."
